## Imports

In [1]:
!pip install datasets
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from huggingface_hub import login

## Huggingface login

In [ ]:
key = None # enter your key here
login(token = key)

## Load Tokenzier and Model

In [3]:
model_name = "meta-llama/Llama-3.2-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto"
)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
print("model loaded successfully")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model loaded successfully


## Prepare Finetuning Dataset

In [4]:
dataset = load_dataset("wikitext", "wikitext-103-v1")
dataset = dataset["test"]

## Preprocess the dataset

In [5]:
def preprocess_data(example):
  tokens = tokenizer(
      example["text"],
      truncation = True,
      padding = "max_length",
      max_length = 512
  )
  tokens["labels"] = tokens["input_ids"].copy()
  return tokens

tokenized_dataset = dataset.map(preprocess_data, batched = True, remove_columns = ["text"])

## Training Arguments

In [6]:
training_args = TrainingArguments(
    output_dir = "./llama3_finetuned",
    save_strategy = "steps",
    learning_rate = 2e-5,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_train_epochs = 1,
    logging_steps = 100,
    save_steps = 500,
    fp16 = True,
    push_to_hub = False
)

## Initialize and Train

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

trainer.train()
results = trainer.evaluate()
print(f"Evaluation results: {results}")

<ipython-input-7-a07543b56a1c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karushsuri (karushsuri-valence-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Decoding

In [10]:
prompt = "Explain the concept of black holes."
inputs = tokenizer(prompt, return_tensors = "pt")
outputs = model.generate(**inputs, max_length = 512)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Explain the concept of black holes. Why are black holes important in modern physics? What is the connection between black holes and the theory of general relativity? Why is black hole formation so important in modern physics?
What is the concept of black hole?
A black hole is a region of space-time that is so dense that no light can escape from it. The event horizon of a black hole is a surface from which nothing can escape, including light. A black hole is a region of space-time where the gravitational pull is so strong that nothing, not even light, can escape from it. The event horizon is the boundary of a black hole. It is a surface from which nothing can escape, including light. A black hole is a region of space-time where the gravitational pull is so strong that nothing, not even light, can escape from it. The event horizon is the boundary of a black hole. It is a surface from which nothing can escape, including light. Black holes are regions of space-time that are so dense that n